# add c_EC as one of the inputs for diffusivity and conductivity

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [2]:
# define function:
def electrolyte_conductivity_Nyman2008_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    """ index = np.piecewise(
        x, 
        [x < 0, ((x >= 0) & (x < 4.541)), x >= 4.541], 
        [lambda x : 0, lambda x : 1/4.541*x, lambda x : 1]) """
    sigma_e = (
        (c_EC <= 4541) * (1/4.541*x) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000)) 
        + (c_EC > 4541) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000))
    )
    return sigma_e
def electrolyte_diffusivity_Nyman2008_wEC(c_e, c_EC,T):
    x = c_EC / c_e
    """ index = np.piecewise(
        x, 
        [x < 0, ((x >= 0) & (x < 4.541)), x >= 4.541], 
        [lambda x : 0, lambda x : 1/4.541*x, lambda x : 1]) """
    D_c_e = (
        (c_EC <= 4541) * (1/4.541*x) * (
        8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10) 
        +  (c_EC > 4541) *  (
        8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10)
    )
    return D_c_e


In [3]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.7e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    "Discharge at 1 C for 130 s", )    ] * 1 )  

In [4]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC"     } )
Model_sdiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "none"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);
Para_0.update({'Outer SEI solvent diffusivity [m2.s-1]':8e-19})

Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = 1.5e-9/1e2;
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   1.5e-9/1e2
Para_0['EC transference number'] =    -1.4
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =  5E-12;
Para_0.update(
    {'Electrolyte diffusivity [m2.s-1]':
    electrolyte_diffusivity_Nyman2008_wEC})
Para_0.update(
    {'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008_wEC})

c_e = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T = Model_ddiff.variables["Cell temperature [K]"]
c_EC = Model_ddiff.variables["EC concentration [mol.m-3]"]
D_e =  Para_0['Electrolyte diffusivity [m2.s-1]'] # Para_0.D_e_dimensional #
sigma_e = Para_0['Electrolyte conductivity [S.m-1]']
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)

turn source term due to Li+ migration anymore
have you come here?


In [5]:
Sim_Landes_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   

Sol_Landes_ddiff    = Sim_Landes_ddiff.solve();


At t = 0.00311049, , mxstep steps taken before reaching tout.
At t = 0.00360209, , mxstep steps taken before reaching tout.
At t = 0.00110208, , mxstep steps taken before reaching tout.
At t = 0.00110178, , mxstep steps taken before reaching tout.


In [10]:
label = ["constant t_0+"] 
output_variables3 = [
    "Terminal voltage [V]",   
    "Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    #"Loss of capacity to SEI [A.h]",
    "Porosity times EC concentration",
    "Porosity times concentration",
    "Electrolyte diffusivity [m2.s-1]",
    "Electrolyte conductivity [S.m-1]"

]
quick_plot = pybamm.QuickPlot([Sol_Landes_ddiff,], output_variables3,label,) #     variable_limits='tight'
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=130.0, step=1.3), Output()), _dom_classes=('…

In [7]:
label = ["constant t_0+"] 
output_variables3 = [
    "Total EC in electrolyte [mol]",
    "Total EC in electrolyte and SEI [mol]",
    "Total lithium in electrolyte [mol]",
]
quick_plot = pybamm.QuickPlot([Sol_Landes_ddiff,], output_variables3,label,) #     variable_limits='tight'
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=130.0, step=1.3), Output()), _dom_classes=('…

In [8]:
Model_new = Model_ddiff.set_initial_conditions_from(
    Sol_Landes_ddiff, inplace=False)
Experiment_s   = pybamm.Experiment( [ (
    "Discharge at 1 C for 1 s", )    ] * 1 )  
Sim_new    = pybamm.Simulation(
    Model_new, experiment = Experiment_s,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   

Sol_new    = Sim_new.solve();

ModelError: initial condition is outside of variable bounds (0, inf) for variable 'Porosity times ec concentration(Negative electrode porosity times EC concentration, Separator porosity times EC concentration, Positive electrode porosity times EC concentration)'.